In [3]:
import datetime
import pytz
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
from dateutil import parser
import matplotlib.pyplot as plt
import time
import re
start_time = time.time()

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import bs4 as bs
import urllib.request
    
#%matplotlib inline

utc=pytz.UTC

#debug = True
debug = True
filepath = './sample_data/'

if debug == False:
    gpx_file = input('Enter file name (must be in same directory as this program for now): ')
    tree = ET.parse(filepath + gpx_file)
else:
    #tree = ET.parse('./sample_data/Exciter_100.gpx')
    tree = ET.parse('./sample_data/S4_Form_Sprints_20_min_M1_Tempo.gpx')
    #tree = ET.parse('See_you_later_Winter_welcome_Spring.gpx')
root = tree.getroot()

each gps device has its own namespace for recording the GPX file... this function reads the embeded namespace

In [4]:
def namespace(element):
    m = re.match('\{(.*)\}', element.tag)
    return m.group(1) if m else ''

In [5]:
#url = 'http://vortex.plymouth.edu/cgi-bin/sfc/gen-statlog-a.cgi?ident=cyxu&pl=rawspec&yy=19&mm=04&dd=04&pg=web'
#dialog http://vortex.plymouth.edu/myo/sfc/statlog-a.html
#choose raw hourly & special METAR obs listings
#but only valid for us/cn/mx
#options
#http://www.ogimet.com/metars.phtml.en
#http://www.ogimet.com/display_metars2.php?lang=en&lugar=sbsp&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=2019&mes=04&day=04&hora=00&anof=2019&mesf=04&dayf=04&horaf=23&minf=59&send=send

#https://www.cyclinganalytics.com/

In [6]:
namespace2 = {'gpx':'http://www.topografix.com/GPX/1/1'}
namespace = {'gpx':namespace(root)}

In [7]:
#helper functions
def deg2rad(x):
    return x * np.pi / 180

def rad2deg(angle):
    return angle*180/np.pi

def kt2ms(speed):
    return speed * 0.5144444

def ms2kmh(speed):
    return speed * 3.6

This function calculates the distance and angle between 2 coordinates

In [8]:
def get_dist_trk(p2, p1):
    #p1 is a list with lat and long in degrees
    #p2 is also a list: lat long
    #returns distance and angle
    #reference: https://www.movable-type.co.uk/scripts/latlong.html
    R = 6371e3 #mean radius of Earth in meters
    phi1 = deg2rad(p1[0])
    phi2 = deg2rad(p2[0])
    lambda1 = deg2rad(p1[1])
    lambda2 = deg2rad(p2[1])
    delta_phi = phi2 - phi1
    delta_lambda = lambda2 - lambda1
    a = np.sin(delta_phi / 2) * np.sin(delta_phi / 2) + \
        np.cos(phi1) * np.cos(phi2) * \
        np.sin(delta_lambda / 2) * np.sin(delta_lambda / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c
    y = np.sin(delta_lambda) * np.cos(phi2)
    x = np.cos(phi1) * np.sin(phi2) - \
        np.sin(phi1) * np.cos(phi2) * np.cos(delta_lambda)
    brg = np.arctan2(y,x) * 180 / np.pi
    return d, brg + 180

Loop through GPX file for each recorded track point and calculate distance and angle

In [41]:
print('Building track database...')
track = pd.DataFrame(columns=['lat', 'lon', 'elev', 'ts', 'dist', 'true_hdg', 'gs', 'delta_t', 'delta_elev'])
#lat in deg decimal
#lon in deg decimal
#elev in meters
#ts in datetime, tz=0
#dist in meters
#true_hdg in degrees
#gs in m/s
trackpoint = {}
debug_counter = 0
for trk in root.findall('gpx:trk', namespace):
    for trksegmt in trk.findall('gpx:trkseg', namespace):
        first_point = True
        trk_pts_number = len(trksegmt.findall('gpx:trkpt', namespace))
        ten_perc = int(len(trksegmt.findall('gpx:trkpt', namespace)) / 10)
        for idx, trkpoint in enumerate(trksegmt.findall('gpx:trkpt', namespace)):
            if (idx % ten_perc) == 0:
                print('{:0.0f}%... '.format(idx/trk_pts_number*100), end='')
            trackpoint['lat'] = float(trkpoint.get('lat'))
            trackpoint['lon'] = float(trkpoint.get('lon'))
            for elev in trkpoint.findall('gpx:ele', namespace):
                #print(elev.text)
                trackpoint['elev'] = float(elev.text)
            for ts in trkpoint.findall('gpx:time', namespace):
                #print(ts.text)
                trackpoint['ts'] = parser.parse(ts.text)
                
            #garmin extensions
            for extensions in trkpoint.findall('gpx:extensions', namespace):
                #print(extensions)
                for ext in extensions:
                    #print(ext.getchildren())
                    #print(ext.__class__)
                    #print('power is: ', ext.tag, ext.text)
                    if 'power' in ext.tag:
                        print('this is power...', float(ext.text))
                    for child in ext:
                        #print(child.tag)
                        if 'atemp' in child.tag:
                            print('this is the ambient temp:', float(child.text))
                        if 'hr' in child.tag:
                            print('this is the heart rate:', float(child.text))
                        if 'cad' in child.tag:
                            print('this is the cadence:', float(child.text))
                            
                    #print(namespace)
                    #for power in ext.findall('gpx:power', namespace):
                    #    print(namespace,power)
                        #trackpoint['m_power'] = float(power.text)

                
                
            if first_point:
                previous_point = trackpoint
                first_point = False
            dist, angle = get_dist_trk([previous_point['lat'],previous_point['lon']], [trackpoint['lat'],trackpoint['lon']])
            if debug_counter < 20:
                #print(dist, trackpoint['lon'], previous_point['lon'])
                debug_counter += 1
            
            trackpoint['delta_t'] = (trackpoint['ts']-previous_point['ts']).total_seconds()
            trackpoint['dist'] = dist
            trackpoint['true_hdg'] = angle
            if trackpoint['delta_t'] != 0:
                trackpoint['gs'] = dist / trackpoint['delta_t']
            else:
                trackpoint['gs'] = 0
                trackpoint['delta_t'] = np.nan
            trackpoint['delta_elev'] = (trackpoint['elev']-previous_point['elev'])
            previous_point = trackpoint
            track = track.append(trackpoint, ignore_index=True)
            trackpoint = {}
end_time = time.time()
print()
print('Track build time was {:1.1f} seconds'.format(end_time - start_time))

Building track database...
0%... this is power... 0.0
this is the ambient temp: 25.0
this is the heart rate: 105.0
this is the cadence: 34.0
this is power... 0.0
this is the ambient temp: 25.0
this is the heart rate: 105.0
this is the cadence: 34.0
this is power... 0.0
this is the ambient temp: 25.0
this is the heart rate: 105.0
this is the cadence: 34.0
this is power... 71.0
this is the ambient temp: 25.0
this is the heart rate: 106.0
this is the cadence: 49.0
this is power... 54.0
this is the ambient temp: 25.0
this is the heart rate: 106.0
this is the cadence: 56.0
this is power... 13.0
this is the ambient temp: 25.0
this is the heart rate: 107.0
this is the cadence: 60.0
this is power... 0.0
this is the ambient temp: 25.0
this is the heart rate: 107.0
this is the cadence: 60.0
this is power... 0.0
this is the ambient temp: 25.0
this is the heart rate: 108.0
this is the cadence: 60.0
this is power... 0.0
this is the ambient temp: 25.0
this is the heart rate: 108.0
this is the cadenc

KeyboardInterrupt: 

In [ ]:
from bs4 import BeautifulSoup

data = """<?xml version="1.0" encoding="UTF-8"?>
<note>
    <to> Tove</to>
    <from>Jani</from>
    <heading>Reminder</heading>
    <body>Don't forget me this weekend!</body>
</note>
"""

soup = BeautifulSoup('./sample_data/S4_Form_Sprints_20_min_M1_Tempo.gpx', 'xml')
print [tag.name for tag in soup.find_all()]